#### Databricks Connector

In [16]:
import pyodbc
import pandas as pd
import pandasql as psql

# Replace <table-name> with the name of the database table to query.
#table_name = "mdp_secure.sams_mbrcard_linking"

# Connect to the Databricks cluster by using the
# Data Source Name (DSN) that you created earlier.
conn = pyodbc.connect("DSN=Databricks_Cluster", autocommit=True)

# Run a SQL query by using the preceding connection.
cursor = conn.cursor()
#cursor.execute(f"SELECT * FROM {table_name} LIMIT 2")

# # Print the rows retrieved from the query.
# print(f"Query output: SELECT * FROM {table_name} LIMIT 2\n")
# for row in cursor.fetchall():
#   print(row)

#### Compute Overall Population

In [5]:
query = (
'''
SELECT DISTINCT A.MEMBERSHIPID, B.CARDNUMBER, CAST(RIGHT(A.CARDNUMBER,9) AS INT) MEMBERSHIP_NBR, A.CARD_NBR, A.CARDSTATUS, B.CARDTYPE, B.CREATEDCLUB, B.SNAPSHOT_BEGIN_TS, 
B.UPDATED_DT,CASE WHEN A.UPDATEDFEATURE IN ('CONVERT_MEMBERSHIP') THEN 1 ELSE 0 END AS EZ_MBR_FLAG,
mem.membershipstatus AS MEMBERSHIP_STATUS,
       mem.membershiptier AS MEMBERSHIP_TIER,
       mem.membershiproledesc AS MEMBERSHIP_ROLE,
       upper(person.firstname) as FIRST_NAME,
       upper(person.lastname) AS LAST_NAME,
       upper(email.emailaddress) AS EMAIL_ADDRESS 
   FROM (SELECT A.*,CAST(LEFT(CARDNUMBER,(LENGTH(CARDNUMBER)-9)) AS INT) AS CARD_NBR FROM mdp_secure.sams_mbrcard_linking A WHERE CURRENT_IND = 'Y') A 
   JOIN (SELECT MEMBERSHIPID, 
                CARDNUMBER,
                CARDTYPE,
                CREATEDCLUB,
                DATE(UPDATEDTS) AS UPDATED_DT,
                CAST(LEFT(CARDNUMBER,(LENGTH(CARDNUMBER)-9)) AS INT) CARD_NBR, 
                DATE(SNAPSHOTBEGINTS) AS SNAPSHOT_BEGIN_TS,
                MIN(SNAPSHOTBEGINTS) MINTS
           FROM mdp_secure.sams_mbrcard_linking 
           WHERE TO_DATE(FROM_UTC_TIMESTAMP(SNAPSHOTBEGINTS,'America/Chicago')) <= CURRENT_DATE()
           AND CURRENT_IND = 'Y'
           GROUP BY 1,2,3,4,5,6, 7)B 
   ON A.MEMBERSHIPID = B.MEMBERSHIPID AND A.CARD_NBR = B.CARD_NBR AND SNAPSHOTBEGINTS = MINTS 
 join mdp_secure.sams_membership mem on A.membershipid = mem.membershipid and mem.current_ind = 'Y'
    join mdp_secure.sams_person person on mem.personid = person.personid and person.current_ind = 'Y'
    join mdp_secure.sams_email_linking email on person.personid = email.personid and email.current_ind = 'Y' and emailstatus = 'ACTIVE' 
   WHERE UPPER(LEFT(TRIM(CARDSTATUS),1)) IN ('A','E','P','D') 
   AND B.CARDTYPE IN ('PREPRINTEDMEMBERSHIPCARD','VIRTUALMEMBERSHIPCARD')
   AND B.CARD_NBR >= 10
   AND A.CARDSTATUS = 'ACTIVE'
   AND MEM.MEMBERSHIPSTATUSCODE IN ('A','E','P')
   AND B.SNAPSHOT_BEGIN_TS >= '2022-05-15' AND B.SNAPSHOT_BEGIN_TS <= '2022-05-21'
   --AND (updatedFeature <> 'HDP_MIGRATION' or updatedFeature is null )
--ORDER BY A.MEMBERSHIP_NBR
'''
)

In [7]:
df = pd.read_sql(query, conn)

In [8]:
df.shape

(1025052, 16)

#### Filtering Base Population

In [11]:
df1 =  psql.sqldf('''select MEMBERSHIPID,
       MEMBERSHIP_NBR,
       CARDNUMBER,
       CARDSTATUS,
       CARDTYPE,
       CREATEDCLUB,
       MEMBERSHIP_STATUS,
       MEMBERSHIP_TIER,
       MEMBERSHIP_ROLE,
       FIRST_NAME,
       LAST_NAME,
       MAX(EMAIL_ADDRESS) AS EMAIL_ADDRESS,
       SNAPSHOT_BEGIN_TS
from df
where cardtype IN ('PREPRINTEDMEMBERSHIPCARD','VIRTUALMEMBERSHIPCARD')
AND MEMBERSHIP_STATUS = 'ACTIVE'
GROUP BY MEMBERSHIPID,
       CARDNUMBER,
       MEMBERSHIP_NBR,
       CARDSTATUS,
       CARDTYPE,
       CREATEDCLUB,
       MEMBERSHIP_STATUS,
       MEMBERSHIP_TIER,
       MEMBERSHIP_ROLE,
       FIRST_NAME,
       LAST_NAME,
       SNAPSHOT_BEGIN_TS''')

In [12]:
df1.shape

(829293, 13)

In [13]:
df1['CARDNUMBER'] = df1['CARDNUMBER'].str[-14:]

In [14]:
df2 = psql.sqldf("select * from df1 order by MEMBERSHIP_NBR")
df2.shape

(829293, 13)

In [15]:
df2.head()

,MEMBERSHIPID,MEMBERSHIP_NBR,CARDNUMBER,CARDSTATUS,CARDTYPE,CREATEDCLUB,MEMBERSHIP_STATUS,MEMBERSHIP_TIER,MEMBERSHIP_ROLE,FIRST_NAME,LAST_NAME,EMAIL_ADDRESS,SNAPSHOT_BEGIN_TS
0,505f76db-06dc-429b-b921-a82fae039071,300,10180000000300,ACTIVE,PREPRINTEDMEMBERSHIPCARD,8194,ACTIVE,PLUS,PRIMARY,DALLAS,COWNE,DCOWNE@MADISON.K12.GA.US,2022-05-20
1,43c6d5d1-23d5-4694-a5aa-36b04b5b40c6,1165,10100000001165,ACTIVE,PREPRINTEDMEMBERSHIPCARD,6456,ACTIVE,CLUB,PRIMARY,KEVIN,FISH,FISHKEVIN1987@GMAIL.COM,2022-05-21
2,2fb5c312-2a46-442c-baf4-2d586b041614,1284,10160000001284,ACTIVE,PREPRINTEDMEMBERSHIPCARD,6644,ACTIVE,CLUB,PRIMARY,DEONA,HALL,DEDEHALL76@YAHOO.COM,2022-05-17
3,eafc10d5-3300-482e-a258-46e248d0c5a2,2306,10910000002306,ACTIVE,PREPRINTEDMEMBERSHIPCARD,8238,ACTIVE,CLUB,COMPLIMENTARY,JACOB,REYNOLDS,REYNOLDSJAKE239@GMAIL.COM,2022-05-17
4,777ab699-3566-4e1c-af4c-dee8a06f63fd,3655,10130000003655,ACTIVE,PREPRINTEDMEMBERSHIPCARD,6382,ACTIVE,CLUB,ADD_ON,JUAN,DONIS,MUNRRAJ12@GMAIL.COM,2022-05-16


#### Exporting Output File

In [10]:
# #Code to split the datafile base on record count

# df_1 = df2.sample(frac = 0.5)
# df_2 = df2.drop(df_1.index)

# df_1.to_csv("Data1.csv")
# df_2.to_csv("Data2.csv")

In [11]:
output1 = "C:/Users/vn537hz/OneDrive - Walmart Inc/Desktop/Sam's Club Files/Adhoc Request/Data_Request_Digital_Membership/OutputFiles/Schedule1.xlsx"
writer = pd.ExcelWriter(output1, engine='xlsxwriter')
df2.to_excel(writer, index=False)
writer.save()